In [1]:
%load_ext autoreload
%autoreload 2
%pdb

Automatic pdb calling has been turned ON


In [15]:
import networkx as nx
import numpy as np
from coldquanta.qiskit_tools.coldquanta_sim_backend import ColdQuantaSimBackend

seed = 666
# We don't have a compiler to the native hardware, so for now we are just going to look at instances of 3-regular graphs
# and compile later.
np.random.seed(seed)
num_qubits = 100
num_graphs_gen = 100
graphs = []

for _ in range(num_graphs_gen):
    graphs.append(nx.generators.random_graphs.random_regular_graph(3, num_qubits))

In [16]:
from networkx.algorithms.isomorphism import is_isomorphic
def isomorphism_classes(graphs):
    classes = []
    for graph in graphs:
        appended = False
        for class_ in classes:
            if is_isomorphic(graph, class_[0]):
                class_.append(graph)
                appended = True
                break
        if not appended:
            classes.append([graph])
    return classes
            

In [17]:
final_graph_num = 10
classes = isomorphism_classes(graphs)
non_isomorphic_graphs = []
for i in range(final_graph_num):
    try:
        non_isomorphic_graphs.append(classes[i][0])
    except IndexError:
        print(f"Only {i} non-isomorphic graphs available.")
        break

KeyboardInterrupt: 

> /Users/anthonypolloreno/repos/networkx/networkx/classes/coreviews.py(78)__getitem__()
     76     __slots__ = ()  # Still uses AtlasView slots names _atlas
     77 
---> 78     def __getitem__(self, name):
     79         return AtlasView(self._atlas[name])
     80 

ipdb> c


In [18]:
non_isomorphic_graphs

In [6]:
from networkx.linalg.graphmatrix import adjacency_matrix
import hashlib
def write_graph(graph, attributes={}):
    h = hashlib.md5()
    arr = adjacency_matrix(graph).toarray()
    h.update(arr)
    hash_ = h.hexdigest()
    try:
        with open(f'{hash_}.pkl', 'rb') as filehandle:
            data = dill.load(filehandle)
            print("Fetching existing file...")
    except FileNotFoundError:
        data = {'graph': graph}
    for k, v in attributes.items():
        if data.get(k) is None:
            data[k] = v
        else:
            print(f"File {hash_}.pkl already has attribute {k}, not overwriting.")
    with open(f'{hash_}.pkl', 'wb') as filehandle:
            data = dill.dump(data, filehandle)

def read_graph(hash_):
    with open(f'{hash_}.pkl', 'rb') as filehandle:
            data = dill.load(filehandle)
    return data

In [7]:
# Gives the same hash on my graphs...
#from networkx.algorithms.graph_hashing import weisfeiler_lehman_graph_hash
import dill 

for graph in non_isomorphic_graphs:
    write_graph(graph)

In [8]:
from classical_optimization.qaoa_circuits import produce_gammas_betas, maxcut_qaoa_circuit, estimate_cost
from qiskit import Aer, execute
from qiskit.providers.aer.extensions import snapshot_density_matrix
from coldquanta.qiskit_tools.modeling.neutral_atom_noise_model import create_noise_model

discretization = 50
max_gamma = 2*np.pi
max_beta = np.pi
gammas, betas = produce_gammas_betas(discretization, max_gamma, max_beta)

/Users/anthonypolloreno/.pyenv/versions/3.7.4/envs/qaoa_venv/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [9]:
def weights(graph):
    rtn = {}
    for e in graph.edges:
        try:
            weight = graph.get_edge_data(e[0], e[1])['weight']
        except KeyError:
            weight = 1
        rtn[e] = weight
    return rtn

In [10]:
def cost(density_matrix, num_qubits, weights):
    rtn = 0
    for edge, weight in weights.items():
        rtn += .5 * weight * (1 - np.trace(Z(*edge, num_qubits).dot(density_matrix)))
    return rtn

def Z(i, j, num_qubits):
    rtn = np.eye(1)
    z = np.array([[1, 0], [0, -1]])
    for k in range(num_qubits):
        if k == i or k == j:
            rtn = np.kron(rtn, z)
        else:
            rtn = np.kron(rtn, np.eye(2))
    return rtn

In [11]:
from classical_optimization.qaoa_circuits import produce_gammas_betas, maxcut_qaoa_circuit, estimate_cost
from qiskit import Aer, execute
from qiskit.providers.aer.extensions import snapshot_density_matrix
from coldquanta.qiskit_tools.modeling.neutral_atom_noise_model import create_noise_model
discretization = 20
max_gamma = 2*np.pi
max_beta = np.pi
gammas, betas = produce_gammas_betas(discretization, max_gamma, max_beta)

In [ ]:
for graph in non_isomorphic_graphs:
    num_qubits = len(graph.nodes)
    simulator = Aer.get_backend('qasm_simulator')
    backend_options = {}
    experiments = []
    for gamma in gammas:
        for beta in betas:
            circuit = maxcut_qaoa_circuit(gammas=[gamma], betas=[beta], p=1, num_qubits=num_qubits, weights=weights(graph), measure=False)
            experiments.append(circuit)

    job = execute(experiments, backend=simulator, backend_options=backend_options)    
    outputs = [result.data.snapshots.density_matrix['output'][0]['value'] for result in job.result().results]
    # The diagonal is real, so we take the first element.
    expectations = [cost(np.array(output)[:, :, 0], num_qubits=num_qubits, weights=weights(graph)) for output in outputs]
    landscape = np.zeros((discretization, discretization))
    for i, gamma in enumerate(gammas):
        for j, beta in enumerate(betas):
            landscape[i][j] = expectations[i*len(betas) + j]
    write_graph(graph, {f"landscape_d{discretization}_b{max_beta}_g{max_gamma}": landscape})
